In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import imblearn
import collections

import sklearn
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report,confusion_matrix

from keras.utils import to_categorical

from keras.utils.vis_utils import plot_model

from collections import Counter
from imblearn.under_sampling import OneSidedSelection
from imblearn.over_sampling import ADASYN

import pickle

Load the Data

In [2]:
csv_file = 'actionstatespaceTStellaULorenzo_combined_final.csv'
dataframe = pd.read_csv(csv_file)

In [3]:
dataframe.head()

#print(dataframe["Time Since Last Charge"].mean())
#print(dataframe["Time Since Last Action Formatted"].mean())

,Action,Last Action,Session Time,Time Since Last Charge,Session Time Formatted,Time Since Last Action,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,...,st1,st2,st3,st4,st5,st6,st7,st8,st9,st10
0,2,17,0:00:55.451657,0.00,0.92,0:00:00.000000,0.00,Happy,1,Low,...,0,0,00:55.451657,00:55,0,0,0,55,0.92,0.92
1,3,2,0:01:29.207295,0.56,1.48,0:00:33.755638,0.55,Happy,1,None,...,0,0,01:29.207295,01:29,1,1,1,29,0.48,1.48
2,8,3,0:01:52.152494,0.95,1.87,0:00:22.945199,0.37,Happy,1,None,...,0,0,01:52.152494,01:52,1,1,1,52,0.87,1.87
3,8,8,0:01:56.868106,1.01,1.93,0:00:04.715612,0.07,Happy,1,None,...,0,0,01:56.868106,01:56,1,1,1,56,0.93,1.93
4,8,8,0:02:01.361565,1.10,2.02,0:00:04.493459,0.07,Happy,1,None,...,0,0,02:01.361565,02:01,2,2,2,1,0.02,2.02


Format and One-hot encode the data

In [4]:
dataframe.drop("tsla1", axis=1, inplace=True)
dataframe.drop("tsla2", axis=1, inplace=True)
dataframe.drop("tsla3", axis=1, inplace=True)
dataframe.drop("tsla4", axis=1, inplace=True)
dataframe.drop("tsla5", axis=1, inplace=True)
dataframe.drop("tsla6", axis=1, inplace=True)
dataframe.drop("st1", axis=1, inplace=True)
dataframe.drop("st2", axis=1, inplace=True)
dataframe.drop("st3", axis=1, inplace=True)
dataframe.drop("st4", axis=1, inplace=True)
dataframe.drop("st5", axis=1, inplace=True)
dataframe.drop("st6", axis=1, inplace=True)
dataframe.drop("st7", axis=1, inplace=True)
dataframe.drop("st8", axis=1, inplace=True)
dataframe.drop("st9", axis=1, inplace=True)
dataframe.drop("st10", axis=1, inplace=True)
dataframe.drop("Session Time", axis=1, inplace=True)
dataframe.drop("Time Since Last Action", axis=1, inplace=True)

dataframe.drop("Session Time Formatted", axis=1, inplace=True)
#dataframe.drop("Time Since Last Charge", axis=1, inplace=True)  

In [5]:
dataframe.head(100)

,Action,Last Action,Time Since Last Charge,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,Input Mood,Activity
0,2,17,0.00,0.00,Happy,1,Low,Negative,Studying
1,3,2,0.56,0.55,Happy,1,None,Negative,Studying
2,8,3,0.95,0.37,Happy,1,None,Negative,Studying
3,8,8,1.01,0.07,Happy,1,None,Negative,Studying
4,8,8,1.10,0.07,Happy,1,None,Negative,Studying
...,...,...,...,...,...,...,...,...,...
95,1,16,4.67,1.10,Happy,1,Low,Neutral,Studying
96,16,1,4.95,0.28,Neutral,0,None,Neutral,Studying
97,3,16,10.38,5.42,Happy,1,Low,Neutral,Studying
98,16,3,10.85,0.47,Happy,1,Low,Neutral,Studying


In [6]:
pd.set_option('display.max_columns', None)
dataframe = pd.get_dummies(dataframe, columns=["Activity"], prefix=["Activity"])
dataframe = pd.get_dummies(dataframe, columns=["Camera Mood"], prefix=["Camera_Mood"])
dataframe = pd.get_dummies(dataframe, columns=["Input Mood"], prefix=["Input Mood"])
#both = pd.concat([dataframe, dum_df])
#dataframe.drop("Input Mood", axis=1, inplace=True) 
#both.head(5)

dataframe = pd.get_dummies(dataframe, columns=["Noise"], prefix=["Noise"])
dataframe = pd.get_dummies(dataframe, columns=["Last Action"], prefix=["Last Action"])
dataframe.head(100)

,Action,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,2,0.00,0.00,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,3,0.56,0.55,1,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8,0.95,0.37,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8,1.01,0.07,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,8,1.10,0.07,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,4.67,1.10,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
96,16,4.95,0.28,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
97,3,10.38,5.42,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
98,16,10.85,0.47,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
X = dataframe.drop('Action', axis=1)
y = dataframe['Action']

In [8]:
X.head(5)

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,0.00,0.00,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.56,0.55,1,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.95,0.37,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.01,0.07,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1.10,0.07,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [9]:
X_train = X[0:222]
X_test = X[222:299]
y_train = y[0:222]
y_test = y[222:299]

In [10]:
X_test

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
222,0.00,0.00,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
223,0.27,0.25,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
224,0.65,0.38,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
225,1.37,0.70,0,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
226,1.70,0.32,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,32.60,0.62,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
295,33.55,0.93,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
296,34.41,0.87,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
297,35.05,0.63,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [11]:
counter = Counter(y_train)
print(counter)

Counter({16: 75, 9: 31, 1: 17, 3: 14, 6: 13, 8: 10, 7: 10, 10: 9, 15: 8, 12: 8, 11: 7, 5: 5, 2: 3, 4: 3, 14: 3, 17: 3, 13: 3})


In [12]:
counter = Counter(y_test)
print(counter)

Counter({16: 28, 2: 8, 1: 6, 15: 6, 6: 5, 9: 5, 10: 3, 3: 3, 8: 2, 12: 2, 5: 2, 7: 2, 11: 2, 17: 1, 13: 1, 4: 1})


Normalize data (train and test done seperately)

In [13]:
X_train_scaled = X_train.copy()

col_names = ['Time Since Last Charge', 'Time Since Last Action Formatted']
features = X_train_scaled[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X_train_scaled[col_names] = features

filename = 'TStellaULorenzo_scaler2.sav'
#pickle.dump(scaler, open(filename, 'wb'))

In [14]:
scaler2 = pickle.load(open(filename, 'rb'))

X_test_scaled = X_test.copy()

features = X_test_scaled[col_names]
features = scaler2.transform(features.values)
X_test_scaled[col_names] = features

X_test_scaled.head()

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
222,-1.478103,-0.580118,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
223,-1.451127,-0.403892,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
224,-1.413160,-0.312254,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
225,-1.341224,-0.086684,0,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
226,-1.308254,-0.354548,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


Load and Train the Model

In [15]:
mlp = MLPClassifier(max_iter=10000, activation='relu', hidden_layer_sizes=(15)) #doesn't converge at 500 so put to higher, do we want it to converge?
mlp

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [16]:
mlp.fit(X_train_scaled,y_train)

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [17]:
#filename = 'TStellaULorenzo_model2.sav'
#pickle.dump(mlp, open(filename, 'wb'))

In [18]:
pred = mlp.predict(X_test_scaled)
pred

array([16, 16, 12,  3,  8,  3, 15, 12, 16, 16, 16,  3, 12, 12,  1, 12, 12,
        9, 16,  3, 16, 16, 12,  8, 15, 15, 16, 15,  8, 16, 16,  3,  9, 16,
        7,  9,  7, 16, 16,  7,  1,  9, 16,  7,  7,  7,  7,  7,  7,  9, 14,
        7, 16,  7,  7, 15,  7, 16,  7,  7,  7,  7, 16,  7,  7,  7,  7,  7,
        7,  7,  6, 16, 16, 17, 16, 16, 17])

In [20]:
y_test.value_counts()

16    28
2      8
1      6
15     6
6      5
9      5
3      3
10     3
5      2
7      2
8      2
11     2
12     2
4      1
13     1
17     1
Name: Action, dtype: int64

In [21]:
from collections import Counter
Counter(pred)

Counter({16: 22,
         12: 7,
         3: 5,
         8: 3,
         15: 5,
         1: 2,
         9: 5,
         7: 24,
         14: 1,
         6: 1,
         17: 2})

In [58]:
y_test.to_csv('actual.csv')

In [107]:
prediction = pd.DataFrame(pred, columns=['predictions']).to_csv('prediction.csv')